In [1]:
import tensorflow as tf
import string
import requests
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

In [29]:
import re 

def remove_urls_nums(text):
    clean_text = re.sub(r'\d+', '', text)
    return url_pat.sub('', clean_text).strip()

In [34]:
print(text[:900])

Baltimore-based
division
of
All..
http://bit.ly/7BnbsR
2009-12-01
12:39:31
76975238
6241426425
Inventions
of
a
Generation:
Think
about
what
you
do
in
the
morning
every
procedure
you
take
part
in
includes
an..
http://bit.ly/638jFN
2009-12-01
11:39:05
76975238
6241422879
Larger
role
for
Chinese
&
Indian
inventors
in
US:
This
is
brought
out
in
a
new
study
published
by
the
Natio..
http://bit.ly/4xaCsa
2009-12-01
11:38:57
76975238
6241421687
No
snow
job:
For
serial
inventor
Arra
David,
the
product
category
was
ripe
for
a
makeover.
He
invented
the
Rebo..
http://bit.ly/4uva1R
2009-12-01
11:38:54
76975238
6239859621
City
firm
sues
gaming
giants:
Eleven
contends
the
wireless
controllers
for
the
Nintendo
Wii,
Sony
Playstation3
..
http://bit.ly/5ixTDr
2009-12-01
10:40:52
76975238
6236589632
Four
Shows
in
the
Next
Two
Weeks?
Sure,
I
Can
Do
That!
|
Atelier
...:
Highlandtown
Holiday
Craft
Show.
Sunday


In [3]:
import os

def split_file(input, output, max_size = 6 * 1024**2):
    os.makedirs(output, exist_ok=True)
    base = os.path.basename(input)
    part = 0
    current_size = 0
    out = None

    with open(input, "rb") as f:
        for line in f:
            if out is None or current_size + len(line) > max_size:
                if out:
                    out.close()
                part += 1
                out_path = os.path.join(output, f"chunk.part{part:d}.txt")
                out = open(out_path, "wb")
                current_size = 0
            out.write(line)
            current_size += len(line)
    if out:
        print(f"Split {input} into {part} parts")
        out.close()
    return part

In [4]:
#split_troll = split_file("combined_gen.txt", "chunks_gen", max_size = 6 * 1024**2 ) 

In [5]:
url_pat = re.compile(r'http\S+', flags = re.IGNORECASE)

with open("chunk.part19.txt", "r", encoding = "utf-8") as t:
    text = t.read()
    
data = text.splitlines()
data = [remove_urls(line) for line in data]

In [6]:
model = load_model("troll_model.keras")

C:\Users\coope\miniforge3\envs\homl3\lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 13 variables whereas the saved optimizer has 24 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


#### LSTM model and train test preparation

In [7]:
tokenizer = Tokenizer(num_words=84294, oov_token="<UNK>")
tokenizer.fit_on_texts(data) #it's going to fit on the data in the forms of lines.

In [8]:
encoded_text=tokenizer.texts_to_sequences(data)

In [9]:
vocab_size = len(tokenizer.word_index) + 1

### Prepare data for training

In [10]:
data_list=[]
for i in encoded_text:
    if len(i)>1:
        for j in range(2,len(i)+1):
            data_list.append(i[:j])
#             print(i[:j]) # if you want to check data

#### Paddding

In [11]:
max_length=25
#max length of line is 25 token per line

In [12]:
sequences=pad_sequences(data_list,maxlen=max_length,padding="pre") # we set the lenght size equal to max_length

In [13]:
X=sequences[:,:-1]
y=sequences[:,-1].astype('int32')

In [14]:
X.shape, y.shape

((226130, 24), (226130,))

In [15]:
seq_length=X.shape[1]

#### Build Model
- We will build a simple LSTM model

In [16]:
model.build(input_shape=(None, seq_length))

In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 24, 50)              │       4,214,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 24, 100)             │          60,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100)                 │          80,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 100)                 │          10,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 84294)               │       8,513,694 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,758,590 (98.26 MB)

 Trainable params: 12,879,294 (49.13 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 12,879,296 (49.13 MB)

In [18]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [19]:
model.fit(X,y,batch_size=256,epochs=300)

Epoch 1/300
884/884 ━━━━━━━━━━━━━━━━━━━━ 196s 218ms/step - accuracy: 0.1015 - loss: 8.2524
Epoch 2/300
884/884 ━━━━━━━━━━━━━━━━━━━━ 150s 169ms/step - accuracy: 0.1280 - loss: 5.0969
Epoch 3/300
884/884 ━━━━━━━━━━━━━━━━━━━━ 149s 168ms/step - accuracy: 0.1329 - loss: 4.6941
Epoch 4/300
884/884 ━━━━━━━━━━━━━━━━━━━━ 150s 170ms/step - accuracy: 0.1385 - loss: 4.5103
Epoch 5/300
884/884 ━━━━━━━━━━━━━━━━━━━━ 147s 166ms/step - accuracy: 0.1412 - loss: 4.3857
Epoch 6/300
884/884 ━━━━━━━━━━━━━━━━━━━━ 143s 162ms/step - accuracy: 0.1434 - loss: 4.2892
Epoch 7/300
884/884 ━━━━━━━━━━━━━━━━━━━━ 144s 163ms/step - accuracy: 0.1453 - loss: 4.2136
Epoch 8/300
884/884 ━━━━━━━━━━━━━━━━━━━━ 144s 163ms/step - accuracy: 0.1460 - loss: 4.1537
Epoch 9/300
884/884 ━━━━━━━━━━━━━━━━━━━━ 147s 166ms/step - accuracy: 0.1472 - loss: 4.1039
Epoch 10/300
884/884 ━━━━━━━━━━━━━━━━━━━━ 148s 167ms/step - accuracy: 0.1478 - loss: 4.0618
Epoch 11/300
884/884 ━━━━━━━━━━━━━━━━━━━━ 149s 168ms/step - accuracy: 0.1496 - loss: 4.02

### Text Generation

In [37]:
text_length = 10

def generate_text(input_text, no_lines):
    general_text = []
    original_input = input_text  # keep the original prefix

    for i in range(no_lines):
        text = []
        for _ in range(text_length):
            encoded = tokenizer.texts_to_sequences([input_text])
            encoded = pad_sequences(encoded, maxlen=seq_length, padding="pre")
            y_pred = np.argmax(model.predict(encoded), axis=-1)

            predicted_word = ""
            for word, index in tokenizer.word_index.items():
                if index == y_pred:
                    predicted_word = word
                    break

            input_text = input_text + ' ' + predicted_word
            text.append(predicted_word)

        line = original_input + " " + " ".join(text)
        general_text.append(line)

        input_text = text[-1]

    return general_text

In [38]:
input_text="me"
text_produced=generate_text(input_text,2)
text_produced

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


['me maybe 2009 10 13 17 10 28 19 37 33',
 'me blogs winter finishing sign idea 7 going to disco balls']

In [39]:
input_text="i want"
text_produced=generate_text(input_text,2)
text_produced

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


['i want to be my las vegas to shared and much what',
 'i want i already it and posting from paranoid shtick 2009 11']

In [23]:
input_text="corrupt"
text_produced=generate_text(input_text,6)
text_produced

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━

['5809230621 <UNK> society women how series who an blake s ass what i wanna got grand catalog u love i did 2009 10 13 09 15 05',
 '00 42 07 topic if moon and blog been facebook cut be 29 nations to are so so 2009 11 17 20 47 31 38 58 46',
 'com taylor approach 360dabestkeptsecret do of the experts good 09 in all permission to bill s 2009 10 05 13 44 11 26 16 27 07 2009',
 '11 17 00 08 07 38 circumcision kee 2009 11 14 13 halloween 43 46 46 30 artist 46 41 50 2009 11 18 06 58 36',
 'yard grand murders staying best lunch at twitteralist 2009 11 30 23 50 2009 11 22 22 28 heavens read the john only his bookstore tune 2009',
 '11 17 00 08 07 38 circumcision kee 2009 11 14 13 halloween 43 46 46 30 artist 46 41 50 2009 11 18 06 58 36']

In [24]:
input_text="love"
text_produced=generate_text(input_text,6)
text_produced

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━

['a was an eyes and wonderful 2009 11 26 19 46 50 46 30 22 46 30 22 46 07 27 22 10 31 16 49 12',
 "00 09 i hate my mind part you she can't choice go thanksgiving anyone at 18 4 in lilwill you going so right cant in the hook",
 'up the last prototyping hdmi robinhood fearless processing huckabee steel are the lord her use checks for hogans increase on more 11 22 05 10 53 00',
 '09 41 48 46 34 bigger jodiekearns night the german 000 white hopes to their today at my floor death in a limited until via account rcschwartzman',
 "07 10 entertainment was me to change its gods day ur am she tortuga gear now always can't the hive extra companies too 2009 11 15 11",
 '00 57 41 00 sports words 8 to figure out 2009 11 08 15 05 46 55 41 30 46 57 48 30 53 16 10 46',
 'com taylor approach 360dabestkeptsecret do of the experts good 09 in all permission to bill s 2009 10 05 13 44 11 26 16 27 07 2009',
 '11 17 00 08 07 38 circumcision kee 2009 11 14 13 halloween 43 46 46 30 artist 46 41 50 2009 11 18 06 

In [25]:
input_text="war is peace"
text_produced=generate_text(input_text,6)
text_produced

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━

['world fight interesting rt a website in the trend run till church mexico been you shovel house any spa business on 2009 11 10 08 13 56',
 "39 41 accepts saints 2009 12 05 15 39 49 10 03 34 00 i'm go 06 18 46 46 30 45 29 54 36 10 06",
 "38 57 41 com does news or blog november i'm lilwill you excited 2009 11 29 20 05 27 24 43 15 08 44 39 36 10",
 "00 07 34 46 blog better love able to death nothing own everybody 2009 12 06 07 39 59 19 43 46 off smu web running can't",
 'd <UNK> lady all the inventions yo genius 5ba nothing 75 as on talking 2009 08 17 17 28 50 48 46 48 48 12 34 46',
 'com taylor approach 360dabestkeptsecret do of the experts good 09 in all permission to bill s 2009 10 05 13 44 11 26 16 27 07 2009',
 '11 17 00 08 07 38 circumcision kee 2009 11 14 13 halloween 43 46 46 30 artist 46 41 50 2009 11 18 06 58 36',
 'yard grand murders staying best lunch at twitteralist 2009 11 30 23 50 2009 11 22 22 28 heavens read the john only his bookstore tune 2009',
 '11 17 00 08 07 38 cir

In [26]:
input_text="freedom is slavery"
text_produced=generate_text(input_text,6)
text_produced

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━

["a side p experience to head a shadows only of grade bring been tonight play work of a i'ma at new google 2009 10 13 12 44",
 '3 25 40 27 39 30 check my what 18 destiny 2009 11 23 19 50 54 43 46 09 16 38 50 51 33 18 03',
 "09 41 46 42 50 40 order 41 46 51 07 42 blog ethics hits trying to i'm show barrowman price content and times while sec bank",
 'coach real versaemerge today signing with throuqh with with were 21 i can macbook x have the money of the city was at did 2009 10 10',
 "00 07 34 46 blog better love able to death nothing own everybody 2009 12 06 07 39 59 19 43 46 off smu web running can't",
 'd <UNK> lady all the inventions yo genius 5ba nothing 75 as on talking 2009 08 17 17 28 50 48 46 48 48 12 34 46',
 'com taylor approach 360dabestkeptsecret do of the experts good 09 in all permission to bill s 2009 10 05 13 44 11 26 16 27 07 2009',
 '11 17 00 08 07 38 circumcision kee 2009 11 14 13 halloween 43 46 46 30 artist 46 41 50 2009 11 18 06 58 36',
 'yard grand murders stayin

In [27]:
input_text="ignorance is strength"
text_produced=generate_text(input_text,6)
text_produced

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━

['swagguhbarbii eightiesbabiesusa com you good m on accident 11 26 famous to show that <UNK> can magic of las 2009 10 10 15 28 21 05 51',
 'conservative friendly witcha what of on luck o 2009 09 16 13 03 19 50 38 39 34 50 technology 10 16 10 56 07 39 35',
 "000 young founders anything to different media you air fries of 140conf a karibou much 01 event come about mtvema disco in the villages 7s you're demand",
 'inside beta 39 thanks s marketing by society any about fail to hanalei pay tutorial to official fail get to more strikes going 2009 11 10 14',
 '00 09 43 46 22 46 46 50 46 46 50 41 28 50 48 48 38 46 18 42 29 29 night 2009 11 16 06',
 "38 57 41 com does news or blog november i'm lilwill you excited 2009 11 29 20 05 27 24 43 15 08 44 39 36 10",
 "00 07 34 46 blog better love able to death nothing own everybody 2009 12 06 07 39 59 19 43 46 off smu web running can't",
 'd <UNK> lady all the inventions yo genius 5ba nothing 75 as on talking 2009 08 17 17 28 50 48 46 48 48 12 34 46',
